In [1]:
from time import sleep
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import urllib
import cchardet
import zipfile
import os


In [6]:
headers={'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) '}
years=list(map(str,list(range(1996,2019))))
years1=list(map(str,list(range(1996,2015))))
years2=list(map(str,list(range(2015,2019))))
dyurl={}
ydlinks={}
baseurl='http://souba-data.com/'

for year in years:
    if year in years1:
        url='http://souba-data.com/k_data/'+year+'_data.htm'  
    if year in years2:
        url='http://souba-data.com/'+year+'_kako_calendar.htm'
    sleep(2)
    r=requests.get(url,timeout=1,headers=headers)
    #エンコード
    r.encoding=cchardet.detect(r.content)['encoding']
    try:
        soup=BeautifulSoup(r.text,'lxml')
    except:
        soup=BeautifulSoup(r.text,'html5lib')
    soup=BeautifulSoup(r.text,'html.parser')
    souptags=soup.find_all('a')
    dlinks=[]
    for souptag in souptags:
        link=souptag.get('href')
        if link != None:
            if (('.zip' in link )or ('.lzh' in link)or ('.xlsx' in link)):
                alink=urllib.parse.urljoin(baseurl,link)
                dlinks.append(alink)
    ydlinks[year]=dlinks

In [10]:
years=list(map(str,list(range(1996,2019))))
rezip=re.compile('.*\.zip')
relzh=re.compile('.*\.lzh')

for year in years:
    dlinks=ydlinks[year]
    foldername=year+'data'
    os.makedirs(foldername,exist_ok=True)
    for dlink in dlinks:
        filename=dlink.split('/')[-1]
        if not os.path.isfile('./'+foldername+'/'+filename):
            sleep(10)
            for ttt in range(3):
                try:
                    file=requests.get(dlink,timeout=2,headers=headers)
                except requests.ReadTimeout as e:
                    sleep(64)
                    print('timeout!')
                else:
                    if file.status_code==200:
                        if not filename in os.listdir('./'+foldername):
                            with open('./'+foldername+'/'+filename,'wb') as f:
                                f.write(file.content)     
                        break
                    elif not file.status_code==404:
                        print('retly',filename)
                        sleep(64)

